In [ ]:
# default_exp core


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def spacifySeq(seq):
  return "".join([ aa +" " for aa in seq]).strip()

In [ ]:
assert spacifySeq("MENDEL") == "M E N D E L"